<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_7/Module_7_01_Recommender_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems (рекомендаційні системи)

![types](https://assets-global.website-files.com/60f03643ffba6a48a3bda298/62690b50cf469a4a8f4f35b7_qj772QmOASv8t2tqRbTaQLmEsnO5dEgD0rIRqsOXh8K8qaCCplZaB2wHTnc5h5oePrXmbW4lLVyrHEI9ybjjBiz3KpmdUl4QNkkY9m3TMvu5IPQngtibC2J4WhKTAk7nXEubAOhq.jpeg)

In [1]:
# !pwd
!pip install "numpy<2.0.0" --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
import pandas as pd

# Source: https://cseweb.ucsd.edu/~jmcauley/datasets.html#clothing_fit

# df = pd.read_json("../../data/modcloth_final_data.json", lines=True)

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
df = pd.read_json("modcloth_final_data.json", lines=True)

In [5]:
df.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82790 entries, 0 to 82789
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_id         82790 non-null  int64  
 1   waist           2882 non-null   float64
 2   size            82790 non-null  int64  
 3   quality         82722 non-null  float64
 4   cup size        76535 non-null  object 
 5   hips            56064 non-null  float64
 6   bra size        76772 non-null  float64
 7   category        82790 non-null  object 
 8   bust            11854 non-null  object 
 9   height          81683 non-null  object 
 10  user_name       82790 non-null  object 
 11  length          82755 non-null  object 
 12  fit             82790 non-null  object 
 13  user_id         82790 non-null  int64  
 14  shoe size       27915 non-null  float64
 15  shoe width      18607 non-null  object 
 16  review_summary  76065 non-null  object 
 17  review_text     76065 non-null 

In [7]:
df = df[~df["user_id"].isna()]  # selects only the rows where user_id is NOT NaN
df = df[~df["item_id"].isna()]
df = df[~df["quality"].isna()]  # rating
df = df[~df["review_text"].isna()]
df = df.reset_index()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76000 entries, 0 to 75999
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           76000 non-null  int64  
 1   item_id         76000 non-null  int64  
 2   waist           2687 non-null   float64
 3   size            76000 non-null  int64  
 4   quality         76000 non-null  float64
 5   cup size        70190 non-null  object 
 6   hips            51599 non-null  float64
 7   bra size        70392 non-null  float64
 8   category        76000 non-null  object 
 9   bust            10894 non-null  object 
 10  height          75001 non-null  object 
 11  user_name       76000 non-null  object 
 12  length          75976 non-null  object 
 13  fit             76000 non-null  object 
 14  user_id         76000 non-null  int64  
 15  shoe size       25810 non-null  float64
 16  shoe width      17279 non-null  object 
 17  review_summary  76000 non-null 

In [9]:
len(df.item_id.unique())

1322

In [10]:
len(df.user_id.unique())

44811

In [11]:
len(df)

76000

## Неперсоналізовані рекомендаційні системи

Popularity-based recommender systems: Popularity-based recommenders work by suggesting the most frequently purchased products to customers. As the name suggests, Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now.

### Frequency of purchase
Popularity-based recommenders work by suggesting the most frequently purchased products to customers. This vague idea can be turned into at least two concrete implementations:
- Check which articles are bought most often across all customers. Recommend these articles to each customer.
Source: https://towardsdatascience.com/how-to-build-popularity-based-recommenders-with-polars-cc7920ad3f68#:~:text=Popularity%2Dbased%20recommenders%20work%20by,these%20articles%20to%20each%20customer.

In [12]:
items_popularity = df.groupby("item_id")["user_id"].count().sort_values(ascending=False)
items_popularity = items_popularity.reset_index()
items_popularity

,item_id,user_id
0,539980,2007
1,668696,1555
2,397005,1506
3,175771,1438
4,407134,1437
...,...,...
1317,542404,1
1318,541405,1
1319,214259,1
1320,536646,1


In [13]:
items_popularity.iloc[:3]["item_id"].to_list()

[539980, 668696, 397005]

In [14]:
import random
popular_items = items_popularity.iloc[:3]["item_id"].to_list()

def present_recommended_products(popular_items: list):
    print("**Currently trending products**")
    print("")

    for index, item_id_ in enumerate(popular_items):
        slice_df = df[df["item_id"] == item_id_]
        print(f"Recommended item {index+1}/{len(popular_items)}: product {item_id_}")

        category = slice_df["category"].unique()[0]
        print(f"{category=}")

        slice_with_reviews = slice_df[~slice_df["review_text"].isna()]
        reviews_for_slice = list(slice_with_reviews["review_text"].unique())
        if len(reviews_for_slice) > 0:
            reviews = random.sample(reviews_for_slice, min(len(reviews_for_slice), 3))
            print(f"User reviews:")
            for review in reviews:
                print("-", review)
            print("...")
        else:
            print("There are no reviews for this product yet.")
        print("")

present_recommended_products(popular_items)

**Currently trending products**

Recommended item 1/3: product 539980
category='tops'
User reviews:
- Love it! The sleeves are just the right length and the material is thick enough to keep its shape over time.
- I got a 2x and it was so huge I exchanged for my usual 1x. Perfection. Comfy cardigan that I wear to death. There is pilling so buy a good electric lint shaver.  Versatile color goes with more than you'd think! Looking to purchase more colors.
- I have a larger chest so I ordered the 1X. When I got the cardigan it was really big though. Perfect as a slouchy, longer cardi with jeans and a tshirt, but it's way to big to wear with dresses and look cute. I ordered the large the next time around and that fit perfectly.
...

Recommended item 2/3: product 668696
category='bottoms'
User reviews:
- great quality
- I love this skirt so much!!! Great quality, fit and I feel so feminine wearing it! Love!!!
- The fabric quality is perfect! The fit is great and the silhouette is perfect for

# Content-based personalized systems

In [15]:
df_reviews = df[["item_id", "review_text", "category"]][~df["review_text"].isna()]
df_reviews.head()

,item_id,review_text,category
0,152702,"I liked the color, the silhouette, and the fab...",new
1,152702,From the other reviews it seems like this dres...,new
2,152702,I love the design and fit of this dress! I wo...,new
3,152702,I bought this dress for work it is flattering...,new
4,152702,This is a very professional look. It is Great ...,new


In [16]:
len(df_reviews)

76000

In [17]:
df_grouped = df_reviews.groupby(["item_id", "category"]).agg({'review_text': ' '.join})
df_grouped = df_grouped.reset_index()
df_grouped.head()

,item_id,category,review_text
0,152702,new,"I liked the color, the silhouette, and the fab..."
1,153494,new,I wanted to fit in this dress so bad so I made...
2,153798,new,Unfortunately the fabric is soooo thin and wri...
3,154411,new,My only complaint is that people notice when I...
4,154882,new,Most of the other reviews said size up one but...


In [18]:
len(df_grouped)

1322

Creating recommendations based on TF-IDF score:

![if-idf2](https://miro.medium.com/v2/resize:fit:860/format:webp/1*dug-uXDMOD6H5JMnYNpgfQ.png)
![tf-idf](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*Uucq42G4ntPGJKzI84b3aA.png)

https://medium.com/@imamun/creating-a-tf-idf-in-python-e43f05e4d424

In [19]:
# TF-IDF Vectorization
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_grouped['review_text'])
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 420695 stored elements and shape (1322, 22088)>

In [20]:
len(df_grouped['review_text'])

1322

In [21]:
pd.DataFrame(tfidf_matrix.toarray())

,0,1,2,3,4,5,6,7,8,9,...,22078,22079,22080,22081,22082,22083,22084,22085,22086,22087
0,0.001489,0.0,0.0,0.0,0.0016,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.001636,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,0.000949,0.0,0.0,0.0,0.0000,0.0,0.001298,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1318,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1319,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1320,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [22]:
tfidf_vectorizer.vocabulary_

{'liked': 10972,
 'the': 19327,
 'color': 4221,
 'silhouette': 17207,
 'and': 1434,
 'fabric': 7073,
 'of': 13143,
 'this': 19449,
 'dress': 6173,
 'but': 3253,
 'ruching': 16247,
 'just': 10353,
 'looked': 11200,
 'bunchy': 3188,
 'ruined': 16261,
 'whole': 21489,
 'thing': 19418,
 'was': 21178,
 'so': 17764,
 'disappointed': 5804,
 'really': 15414,
 'waned': 21119,
 'to': 19692,
 'like': 10971,
 'runs': 16283,
 'little': 11066,
 'small': 17583,
 'would': 21798,
 'need': 12635,
 'size': 17309,
 'up': 20643,
 'make': 11502,
 'it': 10080,
 'workappropriate': 21735,
 'from': 8011,
 'other': 13428,
 'reviews': 15992,
 'seems': 16695,
 'either': 6473,
 'works': 21756,
 'for': 7818,
 'your': 22023,
 'body': 2703,
 'type': 20212,
 'or': 13370,
 'doesn': 5978,
 'have': 8970,
 'waist': 21058,
 'flabby': 7543,
 'tummy': 20099,
 'is': 10058,
 'perfect': 14002,
 'me': 11738,
 'detail': 5615,
 'around': 1702,
 'front': 8017,
 'hides': 9154,
 'everything': 6827,
 'clingyness': 4058,
 'makes': 11507

In [23]:
# Compute cosine similarity between user input and items
user_input = 'Knee-long skirt made of silk'
user_tfidf = tfidf_vectorizer.transform([user_input])
user_df = pd.DataFrame(user_tfidf.toarray())
user_df

,0,1,2,3,4,5,6,7,8,9,...,22078,22079,22080,22081,22082,22083,22084,22085,22086,22087
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
tfidf_vectorizer.vocabulary_["silk"]

17211

In [25]:
user_df[tfidf_vectorizer.vocabulary_["silk"]]

,17211
0,0.726013


In [26]:
tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_["silk"]]

4.855452653939752

Similarity: https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
cosine_similarities

array([0.04200172, 0.04524237, 0.        , ..., 0.02718421, 0.03061408,
       0.04886547])

In [28]:
len(cosine_similarities)

1322

In [29]:
# Get indices of items sorted by similarity
top_n = 3
item_indices = cosine_similarities.argsort()[:-top_n-1:-1]
print("indices:", item_indices)
# Get recommended item names
recommendations = df_grouped['item_id'].iloc[item_indices].tolist()
print("item_ids:", recommendations)

indices: [ 915 1105 1133]
item_ids: [605558, 701811, 714723]


In [30]:
def content_based_recommender(df, user_input, top_n=3):
    """
    Content-based recommender system using TF-IDF.

    Parameters:
    - df: pandas DataFrame with 'item_id' and 'review_text' columns.
    - user_input: textual input representing user preferences.
    - top_n: number of top items to recommend.

    Returns:
    - recommendations: a list of top_n recommended item names.
    """
    # Combine relevant text features into a single string
    df['text_features'] = df['category'] + ' ' + df['review_text']

    # TF-IDF Vectorization
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_features'])

    # Compute cosine similarity between user input and items
    user_tfidf = tfidf_vectorizer.transform([user_input])
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # Get indices of items sorted by similarity
    item_indices = cosine_similarities.argsort()[:-top_n-1:-1]

    # Get recommended item names
    recommendations = df['item_id'].iloc[item_indices].tolist()

    return recommendations

# Example usage
user_preferences = 'Knee-long skirt made of silk'
recommended_products = content_based_recommender(df_grouped, user_preferences, top_n=3)
present_recommended_products(recommended_products)

**Currently trending products**

Recommended item 1/3: product 605558
category='tops'
User reviews:
- Very good quality. The fabric feels like silk. :)
- Just as pictured; soft material; looks and feels good quality. I usually wear an XL or L (I'm an apple shape) and got an L for this item and it fits well.
...

Recommended item 2/3: product 701811
category='bottoms'
User reviews:
- I liked the librarian look, with the plaid and strap details, however on, this skirt was not pretty.  I'm surprised at the cost with how cheaply made this skirt was.  I sent it back.  The length was great though, just wish it was a better quality.
- When I received this skirt, I was pleasantly surprised on the quality of the skirt. The buckle by the waist is a cute feature that makes the skirt unique. My only complaint is that the skirt is too long for my 5'4 stature. It reached about twothree inches above my ankles. Unfortunately, this skirt will be going back.
- The skirt is made very well but is comicall

## Collaborative Filtering (Колаборативні рекомендаційні системи)

## Item-based filtering

Item-based collaborative filtering recommends items based on the similarity between items, particularly their past interactions with users.

In [31]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample dataset (user_id, item_id, rating)
data = {'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
        'item_id': ['A', 'B', 'A', 'C', 'C', 'D', 'B', 'D'],
        'rating': [5, 4, 3, 2, 4, 5, 1, 3]}

df_sample = pd.DataFrame(data)
df_sample

,user_id,item_id,rating
0,1,A,5
1,1,B,4
2,2,A,3
3,2,C,2
4,3,C,4
5,3,D,5
6,4,B,1
7,4,D,3


In [32]:
# Pivot the DataFrame to create a user-item matrix
user_item_matrix = df_sample.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)
user_item_matrix

item_id,A,B,C,D
user_id,,,,
1,5.0,4.0,0.0,0.0
2,3.0,0.0,2.0,0.0
3,0.0,0.0,4.0,5.0
4,0.0,1.0,0.0,3.0


In [33]:
user_item_matrix.T

user_id,1,2,3,4
item_id,,,,
A,5.0,3.0,0.0,0.0
B,4.0,0.0,0.0,1.0
C,0.0,2.0,4.0,0.0
D,0.0,0.0,5.0,3.0


In [34]:
# Calculate cosine similarity between items
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity

array([[1.        , 0.83189033, 0.2300895 , 0.        ],
       [0.83189033, 1.        , 0.        , 0.12478355],
       [0.2300895 , 0.        , 1.        , 0.76696499],
       [0.        , 0.12478355, 0.76696499, 1.        ]])

In [35]:
# Retrieve the target item's similarity scores with all other items
target_item = 'A'
target_item_index = user_item_matrix.columns.get_loc(target_item)
print("item:", target_item, ", index:", target_item_index)
target_item_similarity = item_similarity[target_item_index]
print("item:", target_item, ", similarity scores:", target_item_similarity)

item: A , index: 0
item: A , similarity scores: [1.         0.83189033 0.2300895  0.        ]


In [36]:
# Get indices of items sorted by similarity
top_n = 2
item_indices = target_item_similarity.argsort()[:-top_n-1:-1]
print(f"{item_indices}")

# Get recommended item ids
recommendations = user_item_matrix.columns[item_indices].tolist()
print("Recommended Items:", recommendations)

[0 1]
Recommended Items: ['A', 'B']


In [37]:
def item_based_recommender(df, target_item, top_n=2):
    """
    Item-based collaborative filtering recommender system.

    Parameters:
    - df: pandas DataFrame with 'user_id', 'item_id', and 'rating' columns.
    - target_item: item for which recommendations are sought.
    - top_n: number of top items to recommend.

    Returns:
    - recommendations: a list of top_n recommended items.
    """
    # Pivot the DataFrame to create a user-item matrix
    user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='quality', fill_value=0)  # df["rating"] = df["quality"]
    # Calculate cosine similarity between items
    item_similarity = cosine_similarity(user_item_matrix.T)

    # Retrieve the target item's similarity scores with all other items
    target_item_index = user_item_matrix.columns.get_loc(target_item)
    target_item_similarity = item_similarity[target_item_index]

    # Get indices of items sorted by similarity
    item_indices = target_item_similarity.argsort()[:-top_n-1:-1]

    # Get recommended item ids
    recommendations = user_item_matrix.columns[item_indices].tolist()

    return recommendations

In [38]:
target_item = 714723
recommended_products = item_based_recommender(df, target_item, top_n=4)
present_recommended_products(recommended_products)

**Currently trending products**

Recommended item 1/4: product 714723
category='bottoms'
User reviews:
- The feel, the colours, and the pockets really won me over. I especially love how it flows. That being said, it is a little long for me, and the waist was just a tad too big.
- I'm so glad I bought this skirt! The yellow is the perfect shade without looking too bright or too 70's. The pattern has enough colors in it that it can be matched with tops of many colors. The polyester is heavier than I expected, so it's great for Spring and is completely lined. My only complaint is that the fabric wrinkles fairly easily, but a warm iron takes the wrinkles right out. Keep in mind that this skirt fits at the natural waist, so you may need a small size than you normally wear. My hips are wide, so I normally wear a L, but my waist measurements put me at a M. I ordered the M and it fit perfectly! The length is pretty long, I'm 5'11 and it still hits me below the knee.
- I absolutely love this sk

## User-based filtering

In [39]:
df_sample

,user_id,item_id,rating
0,1,A,5
1,1,B,4
2,2,A,3
3,2,C,2
4,3,C,4
5,3,D,5
6,4,B,1
7,4,D,3


In [40]:
# Pivot the DataFrame to create a user-item matrix
user_item_matrix = df_sample.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity

array([[1.        , 0.64972212, 0.        , 0.19754592],
       [0.64972212, 1.        , 0.34651847, 0.        ],
       [0.        , 0.34651847, 1.        , 0.7407972 ],
       [0.19754592, 0.        , 0.7407972 , 1.        ]])

In [41]:
user_item_matrix

item_id,A,B,C,D
user_id,,,,
1,5.0,4.0,0.0,0.0
2,3.0,0.0,2.0,0.0
3,0.0,0.0,4.0,5.0
4,0.0,1.0,0.0,3.0


In [42]:
item_similarity

array([[1.        , 0.83189033, 0.2300895 , 0.        ],
       [0.83189033, 1.        , 0.        , 0.12478355],
       [0.2300895 , 0.        , 1.        , 0.76696499],
       [0.        , 0.12478355, 0.76696499, 1.        ]])

In [43]:
print("item matrix")
print(user_item_matrix.T)
print("")
print("user matrix")
print(user_item_matrix)

item matrix
user_id    1    2    3    4
item_id                    
A        5.0  3.0  0.0  0.0
B        4.0  0.0  0.0  1.0
C        0.0  2.0  4.0  0.0
D        0.0  0.0  5.0  3.0

user matrix
item_id    A    B    C    D
user_id                    
1        5.0  4.0  0.0  0.0
2        3.0  0.0  2.0  0.0
3        0.0  0.0  4.0  5.0
4        0.0  1.0  0.0  3.0


In [44]:
# Retrieve the target user's similarity scores with all other users
target_user = 1
target_user_index = user_item_matrix.index.get_loc(target_user)
target_user_similarity = user_similarity[target_user_index]

# Get indices of users sorted by similarity
top_n = 3
user_indices = target_user_similarity.argsort()[:-top_n-1:-1]
user_indices

array([0, 1, 3])

In [45]:
user_item_matrix.iloc[user_indices].mean()

,0
item_id,
A,2.666667
B,1.666667
C,0.666667
D,1.000000


In [46]:
# Get recommended item ids based on similar users

#recommendations =
user_item_matrix.iloc[user_indices].mean().sort_values(ascending=False).index.tolist()


['A', 'B', 'D', 'C']

In [47]:
user_item_matrix

item_id,A,B,C,D
user_id,,,,
1,5.0,4.0,0.0,0.0
2,3.0,0.0,2.0,0.0
3,0.0,0.0,4.0,5.0
4,0.0,1.0,0.0,3.0


In [48]:
# def user_based_recommender(df, target_user, top_n=2):
#     """
#     User-based collaborative filtering recommender system.

#     Parameters:
#     - df: pandas DataFrame with 'user_id', 'item_id', and 'rating' columns.
#     - target_user: user for whom recommendations are sought.
#     - top_n: number of top items to recommend.

#     Returns:
#     - recommendations: a list of top_n recommended items.
#     """
#     # Pivot the DataFrame to create a user-item matrix
#     df = df[~df["quality"].isna()]
#     user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='quality', fill_value=0)

#     # Calculate cosine similarity between users
#     user_similarity = cosine_similarity(user_item_matrix)

#     # Retrieve the target user's similarity scores with all other users
#     target_user_index = user_item_matrix.index.get_loc(target_user)
#     target_user_similarity = user_similarity[target_user_index]

#     # Get indices of users sorted by similarity
#     user_indices = target_user_similarity.argsort()[:-top_n-1:-1]

#     # Get recommended item ids based on similar users
#     recommendations = user_item_matrix.iloc[user_indices].mean().sort_values(ascending=False).index.tolist()

#     return recommendations

# # Example usage
# target_user = 320458
# recommended_items = user_based_recommender(df, target_user, top_n=3)
# print("Recommended Items:", recommended_items[:3])

In [50]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

def user_based_recommender(df, target_user, top_n=2):
    """
    User-based collaborative filtering recommender system using sparse matrices.

    Parameters:
    - df: pandas DataFrame with 'user_id', 'item_id', and 'quality' columns.
    - target_user: user ID for whom recommendations are made.
    - top_n: number of similar users to consider.

    Returns:
    - recommendations: list of item_ids recommended for target_user.
    """

    # Фільтруємо NaN у 'quality'
    df = df[~df['quality'].isna()]

    # Створюємо user-item матрицю (пивот)
    user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='quality', fill_value=0)

    if target_user not in user_item_matrix.index:
        raise ValueError(f"Target user {target_user} not found in data")

    # Конвертуємо в sparse матрицю для економії пам'яті
    user_item_sparse = csr_matrix(user_item_matrix.values)

    # Обчислюємо косинусну схожість між користувачами (використовує sparse)
    user_similarity = cosine_similarity(user_item_sparse, dense_output=False)

    # Індекс цільового користувача
    target_idx = user_item_matrix.index.get_loc(target_user)

    # Масив схожості цільового користувача до інших (як numpy array)
    target_similarities = user_similarity[target_idx].toarray().flatten()

    # Відкидаємо самого користувача
    target_similarities[target_idx] = -1

    # Індекси топ-N найбільш схожих користувачів
    top_user_indices = target_similarities.argsort()[-top_n:][::-1]

    # Середній рейтинг цих користувачів по товарах
    recommended_scores = user_item_matrix.iloc[top_user_indices].mean(axis=0)

    # Відсортуємо товари за середнім рейтингом
    recommended_items = recommended_scores.sort_values(ascending=False).index.tolist()

    return recommended_items

# Приклад використання:
target_user = 320458
recommended_items = user_based_recommender(df, target_user, top_n=3)
print("Recommended items:", recommended_items[:3])


Recommended items: [714723, 152702, 590933]


In [51]:
present_recommended_products(recommended_items[:3])

**Currently trending products**

Recommended item 1/3: product 714723
category='bottoms'
User reviews:
- I originally ordered this skirt in a 1x but it was HUGE on me, so I exchanged it for an XL. I probably could've gone down to the L, but I wear this with my shirt tucked it, so it's comfortable. The material feels heavy and high quality, without being too hot.
- The print is beautiful, and the waist fit perfectly, but I didn't find it flattering on my body type (short and curvy, bottom heavy). I'm 5'3 with short legs (27 inseam) and it came right to my lower calf, not a good look. I could have fixed the length by having it shortened by a few inches, but I didn't feel like investing more money in it with the price point. It is very full and pleated throughout which made my already large bottom look bigger.  I would recommend this to someone 5'75'10 with a proportional rear end :)
- The skirt is very high quality and more beautiful in person. Hits right below the knee. Fits as expected

## Hybrid Filtering

In [52]:
# Collaborative Filtering
# collaborative_recommendations = user_based_recommender(df, target_user=320458, top_n=len(df["user_id"].unique()))
collaborative_recommendations = user_based_recommender(df, target_user=320458, top_n=3)

In [53]:
collaborative_recommendations = collaborative_recommendations[:3]

In [54]:
collaborative_recommendations

#  Recommended items: [714723, 589989, 589031]

[714723, 152702, 590933]

In [55]:
# Content-Based Filtering

content_recommendations = content_based_recommender(df_grouped, user_input='Knee-long skirt made of silk', top_n=len(df_grouped["item_id"].unique()))

In [56]:
content_recommendations = content_recommendations[:3]

In [57]:
content_recommendations

[605558, 701811, 714723]

In [58]:
# Hybrid Recommendations (Combining Collaborative and Content-Based)
# Union - https://uk.wikipedia.org/wiki/Об%27єднання_множин
top_n = 3

hybrid_recommendations = set(collaborative_recommendations).union(set(content_recommendations))

hybrid_recommendations = [item for item in hybrid_recommendations]

print("Recommended Items:", hybrid_recommendations[:top_n])


Recommended Items: [714723, 701811, 590933]


In [59]:
present_recommended_products(hybrid_recommendations[:top_n])

**Currently trending products**

Recommended item 1/3: product 714723
category='bottoms'
User reviews:
- LOVE this skirt. Will definitely be wearing it a lot. Superb quality. Definitely will be something you have to iron or steam. Did fit just a tad big in the waist but that's fine with me.
- This skirt fit me exactly right  it was snug but not too tight around my waist and then laid perfectly down to my knees.  The skirt swishes and sways when you walk and is so fun to twirl in.  My only problem with the skirt is that it has a lot a lot a lot of fabric (but of course I don't really mind that because that gives the skirt all the life it has).  This is my new favorite skirt and it's my favorite thing to dance in.
- I loved this skirt but when it arrived it was MILES to big. I measured myself exactly before ordering and the waist it about 5 inches to bog for me. A real disappointment to have to pay to get it to Australia and then pay for a refund.
...

Recommended item 2/3: product 70181

# Matrix Factorization/Singular Vector Decomposition (SVD)

https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/

https://en.wikipedia.org/wiki/Singular_value_decomposition

https://uk.wikipedia.org/wiki/%D0%A1%D0%B8%D0%BD%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D0%B8%D0%B9_%D1%80%D0%BE%D0%B7%D0%BA%D0%BB%D0%B0%D0%B4_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D1%96

In [60]:
import numpy as np

# Sample dataset (user_id, item_id, rating)
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
    'item_id': ['A', 'B', 'A', 'C', 'C', 'D', 'B', 'D'],
    'rating': [5, 4, 3, 2, 4, 5, 1, 3]
}

df_sample = pd.DataFrame(data)

# Convert records into user-book review score matrix
datamatrix = df_sample.pivot(index="user_id", columns="item_id", values="rating").fillna(0)
matrix = datamatrix.values
datamatrix, matrix

(item_id    A    B    C    D
 user_id                    
 1        5.0  4.0  0.0  0.0
 2        3.0  0.0  2.0  0.0
 3        0.0  0.0  4.0  5.0
 4        0.0  1.0  0.0  3.0,
 array([[5., 4., 0., 0.],
        [3., 0., 2., 0.],
        [0., 0., 4., 5.],
        [0., 1., 0., 3.]]))

In [61]:
# Singular value decomposition
u, s, vh = np.linalg.svd(matrix, full_matrices=False)
u, s, vh

(array([[-0.59608532, -0.70833408,  0.25864082, -0.27577173],
        [-0.36286815, -0.17703675, -0.72185025,  0.56206486],
        [-0.65486255,  0.65134711, -0.11519161, -0.36555831],
        [-0.29011038,  0.20656178,  0.63148082,  0.68876713]]),
 array([7.21651241, 6.60170768, 2.98363657, 0.66129963]),
 array([[-0.56385008, -0.37060169, -0.46354614, -0.57432783],
        [-0.61692835, -0.39789319,  0.34102009,  0.58718457],
        [-0.29237698,  0.55839378, -0.63830393,  0.44190516],
        [ 0.46474531, -0.62652357, -0.51127132,  0.36066834]]))

In [62]:
# find the highest similarity
def cosine_similarity(v, u):
  return (v@u)/(np.linalg.norm(v)*np.linalg.norm(u))

In [63]:
vh[:, 0]

array([-0.56385008, -0.61692835, -0.29237698,  0.46474531])

In [64]:
target_item = 0 # A

highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1, vh.shape[1]):
  similarity = cosine_similarity(vh[:, target_item], vh[:, col])
  if similarity > highest_similarity:
    highest_similarity = similarity
    highest_sim_col = col

print(f"Item {highest_sim_col}  (item_id {datamatrix.columns[col]}) is most similar to item {target_item} (item_id {datamatrix.columns[target_item]})")


Item 1  (item_id D) is most similar to item 0 (item_id A)


https://surprise.readthedocs.io/en/stable/matrix_factorization.html

https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

In [65]:
!pip install scikit-surprise

In [66]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [67]:
# Load the dataset into Surprise
reader = Reader(rating_scale=(1, 5))  # df.quality.min()& df.quality.max()
dataset = Dataset.load_from_df(df[['user_id', 'item_id', 'quality']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

# Initialize and train the SVD model
model = SVD(n_factors=100, random_state=42)
# Train the model on the training set
model.fit(trainset)

In [68]:
testset

[(515984, 739127, 4.0),
 (529649, 427041, 3.0),
 (209767, 427567, 5.0),
 (881136, 407134, 3.0),
 (538513, 407134, 3.0),
 (949604, 539980, 4.0),
 (5544, 647235, 2.0),
 (858201, 340237, 4.0),
 (476561, 693560, 2.0),
 (591404, 646088, 3.0),
 (622441, 407134, 4.0),
 (779253, 165716, 2.0),
 (510371, 657242, 3.0),
 (682499, 715662, 2.0),
 (384874, 407134, 4.0),
 (685062, 757242, 4.0),
 (457121, 427567, 5.0),
 (741537, 806856, 5.0),
 (887037, 507565, 3.0),
 (947324, 165716, 1.0),
 (372025, 803464, 1.0),
 (498413, 486643, 2.0),
 (516990, 380801, 5.0),
 (874483, 391519, 3.0),
 (685752, 359893, 5.0),
 (260839, 519836, 4.0),
 (16520, 169727, 4.0),
 (496365, 338596, 5.0),
 (750423, 410263, 4.0),
 (256600, 210299, 3.0),
 (813873, 659701, 5.0),
 (816475, 412737, 4.0),
 (289306, 427567, 3.0),
 (753687, 486643, 4.0),
 (457322, 153494, 3.0),
 (308738, 298256, 4.0),
 (220838, 407574, 4.0),
 (552053, 645822, 5.0),
 (57577, 719701, 2.0),
 (850000, 416738, 4.0),
 (304621, 796383, 2.0),
 (329621, 639328, 4.

In [73]:
# Make predictions on the test set
predictions = model.test(testset)

# recommend items for a specific user
target_user = 320458
items_to_recommend = df['item_id'].unique()

# Create a list of tuples (item_id, predicted_rating)
user_predictions = [(item, model.predict(target_user, item).est) for item in items_to_recommend]

# Sort the predictions by the predicted rating in descending order
# sorted_predictions = user_predictions.sort(key=lambda x: x[1], reverse=True)
sorted_predictions = sorted(user_predictions, key=lambda x: x[1], reverse=True)

# Print the top recommended items
top_n = 3
print(f"Top {top_n} recommended items for user {target_user}:")

# for i in range(top_n):
#   print(f"Item ID: {sorted_predictions[i][0]}, Predicted Rating: {sorted_predictions[i][1]:.2f}")
# print(" ")
for item_id, predicted_rating in sorted_predictions[:top_n]:
    print(f"Item ID: {item_id}, Predicted Rating: {predicted_rating:.2f}")


Top 3 recommended items for user 320458:
Item ID: 768644, Predicted Rating: 4.85
Item ID: 680903, Predicted Rating: 4.83
Item ID: 697881, Predicted Rating: 4.73


In [74]:
top_n_recommendations = [item for item, _ in sorted_predictions[:top_n]]
print("Recommended Items:", top_n_recommendations)

Recommended Items: [768644, 680903, 697881]


In [75]:
present_recommended_products(top_n_recommendations)

**Currently trending products**

Recommended item 1/3: product 768644
category='bottoms'
User reviews:
- Love this skirt. :)
- I have this skirt in navy, wine and forest.. I absolutely love them! They are so versatile, flattering and are of high quality!
- This skirt should cost much more; it's very well made. The pocket is a single pocket with two entrances, like in a hoodie. The only thing wrong with it is that it's too thick and heavy a material to wear in the hot area I live. I wish I could count on traveling somewhere cooler so I'd have a chance to wear it, but, alas, that is not the case. I still haven't quite put it back in the box yet, though  it really is a nice skirt!
...

Recommended item 2/3: product 680903
category='bottoms'
User reviews:
- This skirt is fantastic. Very comfortable to wear and flattering. I will definitely be getting this skirt in additional colors.
- Very high quality skirt. Flattering cut and the sizing was perfect. However, the color wasn't exactly as I